In [28]:
import pandas as pd
import numpy as np

In [29]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [74]:
df = pd.read_csv(r"C:\Users\hp\OneDrive\codes\python\Learn_ML\1. Datasets\covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [75]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.3,random_state=42)

### Naive Approach

#### for city and gender we can do OneHotEncoding because it does not follow any order , for cough we need to do OrdinalEncoding and for output hascovid we can do LabelEncoding

In [55]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [56]:

X_train

,age,gender,fever,cough,city
23,80,Female,98.0,Mild,Delhi
21,73,Male,98.0,Mild,Bangalore
35,82,Female,102.0,Strong,Bangalore
50,19,Male,101.0,Mild,Delhi
25,23,Male,NaN,Mild,Mumbai
...,...,...,...,...,...
27,33,Female,102.0,Strong,Delhi
9,64,Female,101.0,Mild,Delhi
80,14,Female,99.0,Mild,Mumbai
56,71,Male,NaN,Strong,Kolkata


In [57]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [59]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [61]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [62]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [63]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [65]:
X_train_transformed

array([[ 80.        ,  98.        ,   0.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 73.        ,  98.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 82.        , 102.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 19.        , 101.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 23.        , 100.79452055,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 65.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 44.        , 104.        ,   1.        ,   0.        ,
          0.        ,   1.        ,   0.        ],
       [ 14.        , 101.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 71.        ,  98.        ,   0.        ,   0.        ,
          1.    

#### Better Approach


In [87]:
from sklearn.compose import ColumnTransformer

In [92]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

# passthrough means the other columns will not be changed ina any way but had it been drop , age column would have been dropped

In [93]:
transformer.fit_transform(X_train).shape

(70, 7)

In [94]:
transformer.transform(X_test).shape

(30, 7)